## Some words on this notebook

Under the pretext of 'analysing' a bit the EEG files, I created this notebook which in reality is more a playground for me to get familiar with the data. For this prupose I wanted to investigate how to extract the information from the .mat files, how to plot the data of the EEG files, etc.
I also produced some artificial data and tried to find patterns that could be used to make a prediction.
I also wanted to see if it was possible to use the image of an espectogram for the same purposes.
All this without understanding really anything about EEG.

After a bit of research I realized it could have been interesting to know what kind of montage was used to produce these EEG values, so that we could train an algorithm to identify which electrodes (and the value differences between them) are more likely to yield a good prediction. However, after looking at the EEG plot (below) it seems that the EEGs were produced using one common reference for all electrode measurements.


So, I don't want to spend too much time on getting educated on the vast world of neuro science interpretations. Since I wasn't able to -very- easily find something visually useful, I will stop this notebook and instead I will go directly into trying to train an RNN algorithm for predictions.

In [ ]:
!rm -r '/kaggle/working/train/'

In [ ]:
#Visualizing an example:
from scipy.io import loadmat
import os
import numpy as np
import matplotlib.pyplot as plt
interictal_tst = '/kaggle/input/seizure-prediction/Patient_1/Patient_1/Patient_1_interictal_segment_0001.mat'
interictal_data = loadmat(interictal_tst)          # loads the file as a dictionary
for item in interictal_data.items():
    print(item)

In [ ]:
segment_tst = '/kaggle/input/seizure-prediction/Patient_1/Patient_1/Patient_1_test_segment_0001.mat'
segment_data = loadmat(segment_tst)          # loads the file as a dictionary
for item in segment_data.items():
    print(item)

In [ ]:
files= [x for x in os.listdir('/kaggle/working/Patient1/train/preictal/')]
files_i=[x for x in os.listdir('/kaggle/working/Patient1/train/interictal/')]
len(files),len(files_i)

In [ ]:
interictal_data.get('interictal_segment_1')

Check for data inconsistencies, like for example, that there are as much data samples needed as data seconds and that these two correspond to the data samples per seconds.

In [ ]:
# search_key = "_segment_"       # search key string

# for dirname, _, filenames in os.walk('/kaggle/input/seizure-prediction/'):
#     for filename in filenames:
#         if filename != "sampleSubmission.csv":
#             try:
#                 segment_dict = loadmat(os.path.join(dirname, filename))        # loads the .mat file as a dictionary
#                 segment = dict(filter(lambda item: search_key in item[0], segment_dict.items()))  # retrieves only the item which contain the data of interes
#                 segment_list_of_values = list(segment.values())
#                 total_num_dataSamples = np.array(segment_list_of_values[0][0][0][0].shape[1], dtype=np.float)
#                 datarows_duration_secs = np.array(segment_list_of_values[0][0][0][1], dtype=np.float)
#                 datasamples_perSecond = np.array(segment_list_of_values[0][0][0][2], dtype=np.float)

#                 if total_num_dataSamples != np.around((datarows_duration_secs * datasamples_perSecond)):
#                     print("Data inconsistency in file: %s" % os.path.join(dirname, filename))
#             except:
#                 e = sys.exc_info()[0]
#                 print("Exception %s caught on file %s " % (e, os.path.join(dirname, filename)))
#                 break
# print("Check done.")

Does all the files of the same patient or dog (i.e. Patient_1 or Patient_2 or Dog_1, etc.) have the same number of electrodes?
And do they all have the same names?

In [ ]:
# search_key = "_segment_"       # search key string
# subdirs = [
#     '/kaggle/input/seizure-prediction/Patient_1/Patient_1',
#     '/kaggle/input/seizure-prediction/Patient_2/Patient_2',
#     '/kaggle/input/seizure-prediction/Dog_1/Dog_1',
#     '/kaggle/input/seizure-prediction/Dog_2/Dog_2',
#     '/kaggle/input/seizure-prediction/Dog_3/Dog_3',
#     '/kaggle/input/seizure-prediction/Dog_4/Dog_4',
#     '/kaggle/input/seizure-prediction/Dog_5/Dog_5',
# ]

# for dirname in subdirs:
#     for dirname, _, filenames in os.walk(dirname):
#         # auxiliar variables needed to monitor both the
#         # number of electrodes and electrode names between
#         # the last file analyzed and the current file analyzed
#         numElectrodes_lastFile = 0
#         firstComparisonDone = False
#         lastFileName = ""
#         electrodeNames_lastFile = []
#         for filename in filenames:
#             matFile = scipy.io.loadmat(os.path.join(dirname, filename))
#             segment = dict(filter(lambda item: search_key in item[0], matFile.items()))  # retrieves only the item which contain the data of interes
#             segment_list_values = list(segment.values())
#             # number of electrodes on the currently analyzed file
#             numElectrodes_currentFile = segment_list_values[0][0][0][3][0].shape[0]
#             # next two lines of code are relevant only if during the
#             # first cycle of this loop
#             numElectrodes_lastFile = numElectrodes_currentFile if True is not firstComparisonDone else numElectrodes_lastFile
#             firstComparisonDone = True if True != firstComparisonDone else firstComparisonDone
#             # check if number of electrodes between the current file and the last
#             # file analyzed are equal
#             if numElectrodes_currentFile != numElectrodes_lastFile:
#                 print("Discrepancy in number of electrodes between files:")
#                 print("     Last file    %s with %d electrodes" % (lastFileName, numElectrodes_lastFile))
#                 print("     Current file %s with %d electrodes" % (filename, numElectrodes_currentFile))
#             numElectrodes_lastFile = numElectrodes_currentFile
#             # collect the name of the electrodes on the current file being analyzed
#             electrode_names = [name[0] for name in segment_list_values[0][0][0][3][0]]
#             if len(electrodeNames_lastFile) > 0:
#                 # check if the electrode names of the last file analyzed and 
#                 # the current file analyzed are equal
#                 commonNames = set(electrodeNames_lastFile) & set(electrode_names)
#                 if len(commonNames) != len(electrode_names):
#                     print("Different electrode names on files: ")
#                     print("     %s (last faile name)" % lastFileName)
#                     print("         Electrode names: ", electrodeNames_lastFile)
#                     print("     %s " % filename)
#                     print("         Electrode names: ", electrode_names)
#             electrodeNames_lastFile = electrode_names
#             lastFileName = filename
# print("Check done")

Here's an example of how an eeg looks like.

In [ ]:
# Code in this cell was partially taken from:
# https://matplotlib.org/3.3.1/gallery/specialty_plots/mri_with_eeg.html

import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import numpy as np

file_to_inspect = loadmat('/kaggle/input/seizure-prediction/Patient_1/Patient_1/Patient_1_interictal_segment_0001.mat')
# retrieves only the item which contain the data of interese
search_key = '_segment_'
segment = dict(filter(lambda item: search_key in item[0], file_to_inspect.items()))
segment = list(segment.values())
data = segment[0][0][0][0]
num_electrodes = data.shape[0]             # rows (i.e. electrodes) in the data matrix
n_samples = data.shape[1]                  # number of samples on each row (i.e. electrode's samples)
electrode_names = segment[0][0][0][3][0]   # name or labels of the electrodes
t = 10 * np.arange(n_samples) / n_samples

# create the 'figure'
fig = plt.figure("EEG samples",figsize=(30,15))

ticklocs = []
ax = fig.add_subplot(1, 1, 1)
ax.set_xlim(0, 10)
ax.set_xticks(np.arange(10))
data_min = data.min()
data_max = data.max()
dr = (data_max - data_min) * 0.7   #crowd it a bit
y0 = data_min
y1 = (num_electrodes -1) * dr + data_max
ax.set_ylim(y0, y1)

segs = []
for electrode in range(num_electrodes):
    segs.append(np.column_stack((t, data[electrode, :])))
    ticklocs.append(electrode * dr)
    
offsets = np.zeros((num_electrodes, 2), dtype=float)
offsets[:, 1] = ticklocs

lines = LineCollection(segs, offsets=offsets, transOffset=None)
ax.add_collection(lines)

# Set the yticks to use axes coordinates on the y axis
ax.set_yticks(ticklocs)
ax.set_yticklabels(electrode_names)

ax.set_xlabel('Time (minutes)')

plt.show()

In [ ]:
import pandas as pd
import numpy as np
import copy
import scipy.signal as signal
import scipy.stats as stats
import scipy.io as sio
import tqdm

class Dataset:
    def __init__(self,path):
        self.path = path
        if self.path[-1] != '/':
            self.path += '/'
        self.files = np.concatenate([[self.path+'/interictal/'+x for x in os.listdir(self.path+'/interictal/')], [self.path+'/preictal/'+x for x in os.listdir(self.path+'/preictal/')]])
        self.N_interictal = 15000
        self.N_preictal = 5400
        self.targets=np.array([0 if x < self.N_interictal else 1 for x in range(len(self.files))])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, item):
        data = np.load(self.files[item])
        target = self.targets[item]
        data = np.expand_dims(data,axis=0)
        return data,target

    def split_random(self,N_valid):
        sample = np.random.choice(len(self), len(self), replace=False)
        self.files, self.targets = self.files[sample], self.targets[sample]
        train = copy.deepcopy(self)
        valid = copy.deepcopy(self)
        train.files, train.targets = train.files[N_valid:], train.targets[N_valid:]
        valid.files, valid.targets = valid.files[:N_valid], valid.targets[:N_valid]
        return train,valid

In [ ]:
def epoch_raw(raw_path, secs=30):
    raw_files_interictal = sorted([f for f in os.listdir(raw_path) if 'interictal' in f])
    raw_files_preictal = sorted([f for f in os.listdir(raw_path) if 'preictal' in f])
    raw_files_test = sorted([f for f in os.listdir(raw_path) if 'test' in f])
    raw_files = {'interictal': raw_files_interictal, 'preictal': raw_files_preictal, 'test':raw_files_test}
    pat_dir = '/kaggle/working/Patient1/'
    train_dir = pat_dir+'/train/'
    test_dir = pat_dir+'/test/'
    train_dir_interictal = train_dir+'/interictal/'
    train_dir_preictal = train_dir+'/preictal/'
    if not os.path.isdir(pat_dir):
        os.mkdir(pat_dir)
    if not os.path.isdir(train_dir):
        os.mkdir(train_dir)
    if not os.path.isdir(test_dir):
        os.mkdir(test_dir)
    if not os.path.isdir(train_dir_interictal):
        os.mkdir(train_dir_interictal)
    if not os.path.isdir(train_dir_preictal):
        os.mkdir(train_dir_preictal)
    data_dir = {'interictal': train_dir_interictal, 'preictal': train_dir_preictal, 'test':test_dir}
    
    for category in ['interictal','preictal', 'test']:
        for _,f in enumerate(raw_files[category]):
            search_key = "_segment_"       # search key string
            try:
                matFile = loadmat(os.path.join(raw_path,f))
            except OSError as error:
                print(error)
            segment = dict(filter(lambda item: search_key in item[0], matFile.items()))
            segment_list_values = list(segment.values())
            x = np.array(segment_list_values[0][0][0][0], dtype=np.float)
            Fs = np.array(segment_list_values[0][0][0][2], dtype=np.float)[0][0]
            L = int(Fs*secs)
            for electrode in range(x.shape[0]):
                for i in range(L,x.shape[1]+1,L):
                    digit = f'_{electrode}_{int(i//L)}'
                    save_filepath = data_dir[category]+f[:-4]+digit+'.npy'
                    spect,_,_,_ = plt.specgram(x[electrode, i-L:i], NFFT=1024, Fs=5000, noverlap=128)
                    spect = stats.zscore(spect[:200,:],axis=1)
                    try:
                        np.save(save_filepath, spect)
                    except OSError as e:
                        print(e)
            print(f"Finished file {f}")
        print(f"Finished category {category}")   
    return

patient_dir = '/kaggle/input/seizure-prediction/Patient_1/Patient_1'
epoch_raw(patient_dir, 30)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class NN(nn.Module):
    def __init__(self,NFILT=256,NOUT=2):
        super(NN,self).__init__()
        self.conv0 = nn.Conv2d(1,NFILT,kernel_size=(200,3),padding=(0,1),bias=False)
        self.bn0 = nn.BatchNorm2d(NFILT)
        self.gru = nn.GRU(input_size=NFILT,hidden_size=128,num_layers=1,batch_first=True,bidirectional=False)
        self.fc1 = nn.Linear(128,NOUT)



    def forward(self, x):
        x = F.relu(self.bn0(self.conv0(x)))
        x = x.squeeze().permute(0,2,1)
        x,_ = self.gru(x)
        x = F.dropout(x,p=0.5,training=self.training)
        x = self.fc1(x)
        return x

In [ ]:
import torch
import numpy as np
from scipy.special import softmax,expit
from sklearn.metrics import f1_score,confusion_matrix,cohen_kappa_score,roc_curve,roc_auc_score,average_precision_score


class Statistics(object):
    def __init__(self):
        self.target = []
        self.logits = []

    def reset(self):
        self.target = []
        self.logits = []

    @staticmethod
    def idx2onehot(idx_array):
        y = np.zeros((idx_array.shape[0], idx_array.max() + 1))
        y[np.arange(y.shape[0]), idx_array] = 1
        return y

    @staticmethod
    def F1(conf):
        x0 = np.sum(conf, 0)
        x1 = np.sum(conf, 1)
        dg = np.diag(conf)
        f1 = 2 * dg / (x0 + x1)
        return f1

    @staticmethod
    def Kappa(conf):
        x0 = np.sum(conf, 0)
        x1 = np.sum(conf, 1)
        N = np.sum(np.sum(conf))
        ef = np.sum(x0 * x1 / N)
        dg = np.sum(np.diag(conf))
        K = (dg - ef) / (N - ef)
        return K

    def append(self,target,logits):
        self.logits.append(logits.data.cpu().numpy())
        self.target.append(target.data.cpu().numpy())

    @staticmethod
    def random_auprc(target):
        y_chance = np.zeros((target.max()+1,))
        for i in range(target.max()+1):
            y_chance[i] = len(target[target==i]) / len(target)

        return y_chance


    def evaluate(self):
        self.logits = np.concatenate(self.logits)
        self.target = np.concatenate(self.target).astype('int32')

        self.probs = softmax(self.logits,axis=1)
        self.argmax = np.argmax(self.probs,axis=1)

        CONF = np.array(confusion_matrix(y_true=self.target,y_pred=self.argmax))
        F1 = Statistics.F1(CONF)
        KPS = Statistics.Kappa(CONF)
        AUROC = roc_auc_score(y_true=Statistics.idx2onehot(self.target),y_score=self.probs,average=None)
        AUPRC = average_precision_score(y_true=Statistics.idx2onehot(self.target),y_score=self.probs,average=None)
        AUPRC_chance = self.random_auprc(self.target)

        print(CONF)
        print(F1)
        print(KPS)
        print(AUROC,np.mean(AUROC))
        print(AUPRC,np.mean(AUPRC))
        print(AUPRC_chance)

        self.reset()

In [ ]:
import torch
from torch.utils.data import DataLoader

N = 50*300+18*300
train_dataset, valid_dataset = Dataset('/kaggle/working/Patient1/train/').split_random(int(0.3*N))
print("N Train, valid: ", len(train_dataset),len(valid_dataset))
NWORKERS = 24
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

TRAIN = DataLoader(dataset=train_dataset,
                   batch_size=32,
                   shuffle=True,
                   drop_last=False,
                   num_workers=NWORKERS)

VALID = DataLoader(dataset=valid_dataset,
                   batch_size=32,
                   shuffle=True,
                   drop_last=False,
                   num_workers=NWORKERS)


model = NN(2).to(DEVICE)
optimizer = optim.Adam(model.parameters(),lr=1e-3,weight_decay=1e-4)
loss = nn.CrossEntropyLoss()
statistics = Statistics()
N_epochs = 30
epoch_loss={'train':np.zeros((N_epochs)),'val':np.zeros((N_epochs))}
for epoch in range(N_epochs):
    model.train()
    for i,(x,t) in enumerate(TRAIN):
        optimizer.zero_grad()
        x = x.to(DEVICE).float()
        t = t.to(DEVICE).long()
        y = model(x)
        J = loss(input=y[:,-1,:],target=t)
        epoch_loss['train'][epoch] += J.data.cpu().numpy()
        J.backward()
        optimizer.step()

        if i%50==0:
            print('EPOCH:{}\tITER:{}\tLOSS:{}'.format(str(epoch).zfill(2),
                                                      str(i).zfill(5),
                                                      J.data.cpu().numpy()))
    epoch_loss['train'][epoch]/=len(TRAIN)
        
    # evaluate results for validation set
    model.eval()
    for i,(x,t) in enumerate(VALID):
        x = x.to(DEVICE).float()
        t = t.to(DEVICE).long()
        y = model(x)
        val_loss = loss(input=y[:,-1,:],target=t)
        epoch_loss['val'][epoch] += val_loss.data.cpu().numpy()
        statistics.append(target=t,logits=y[:,-1,:])
    epoch_loss['val'][epoch]/=len(VALID)
    statistics.evaluate()

In [ ]:
fig=plt.figure()
plt.title("Epoch Loss")
plt.plot(epoch_loss['train'])
plt.plot(epoch_loss['val'])
plt.legend(['train', 'val'])
plt.show()

In [ ]:
torch.save(model, "/kaggle/working/Patient1/cnngru.pth")

In [ ]:
def saveSpectrograms(file, electrode):
    search_key = "_segment_"       # search key string
    matFile = loadmat(os.path.join(file))
    segment = dict(filter(lambda item: search_key in item[0], matFile.items()))
    segment_list_values = list(segment.values())
    x = np.array(segment_list_values[0][0][0][0][electrode], dtype=np.float)
    Fs = np.array(segment_list_values[0][0][0][2], dtype=np.float)[0][0]
    L = int(Fs*3)
    NFFT = 1024  # the length of the windowing segments
    for i in np.arange(L,x.shape[0]+1,L):
        _,_, data = signal.spectrogram(x[i-L:i], fs=Fs,nperseg=256,noverlap=128,nfft=NFFT)
        np.save()


def plotSpectograms(file, electrode=0):
    search_key = "_segment_"       # search key string
    matFile = loadmat(os.path.join(file))
    segment = dict(filter(lambda item: search_key in item[0], matFile.items()))
    segment_list_values = list(segment.values())

    x = np.array(segment_list_values[0][0][0][0][electrode], dtype=np.float)
    x=x[:15000]
    #secs = np.array(segment_list_values[0][0][0][1], dtype=np.float)
    Fs = np.array(segment_list_values[0][0][0][2], dtype=np.float)[0][0]
    print("Fs,",Fs)
    NFFT = 1024  # the length of the windowing segments
    dt = 1/Fs
    t = np.arange(0.0, 3.0, dt)

    fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(15,10))
    ax1.plot(t, x)
    Pxx, freqs, bins, im = ax2.specgram(x, NFFT=NFFT, Fs=Fs, noverlap=128)
    print(Pxx.shape)
    print(freqs.shape)
    # The `specgram` method returns 4 objects. They are:
    # - Pxx: the periodogram
    # - freqs: the frequency vector
    # - bins: the centers of the time bins
    # - im: the .image.AxesImage instance representing the data in the plot
    #plt.show()
    
file = [
    '/kaggle/input/seizure-prediction/Patient_1/Patient_1/Patient_1_interictal_segment_0001.mat',
    '/kaggle/input/seizure-prediction/Patient_1/Patient_1/Patient_1_preictal_segment_0001.mat',
        '/kaggle/input/seizure-prediction/Patient_1/Patient_1/Patient_1_preictal_segment_0011.mat',
    '/kaggle/input/seizure-prediction/Patient_1/Patient_1/Patient_1_interictal_segment_0023.mat'

    #'/kaggle/input/seizure-prediction/Dog_3/Dog_3/Dog_3_interictal_segment_0005.mat',
   # '/kaggle/input/seizure-prediction/Dog_3/Dog_3/Dog_3_preictal_segment_0005.mat'
]
for f in file:
    plotSpectograms(f, electrode=5)
# 513, 3348

In [ ]:
search_key = "_segment_"       # search key string
subdirs = [
    '/kaggle/input/seizure-prediction/Patient_1/Patient_1'
]

for dirname in subdirs:
    for dirname, _, filenames in os.walk(dirname):
        print(len(filenames))
        print("N test:", len([f for f in filenames if "test" in f]))
        print("N interictal:", len([f for f in filenames if "interictal" in f]))
        print("N preictal:", len([f for f in filenames if "preictal" in f]))
        numElectrodes_lastFile = 0
        firstComparisonDone = False
        lastFileName = ""
        electrodeNames_lastFile = []
        for filename in filenames:
            print(filename)

In [ ]:
!pwd